In [60]:
import pymongo
import math 
import re
import pandas as pd
import numpy as np
import pylab 
from matplotlib import pyplot as plt
from scipy import stats 
# from  qgrid import show_grid as show 
from datetime import datetime
import seaborn as sns

def gettimetamps(containerId): 
    containerdata=list(sensors.find({'target':containerId},projection=['timestamp']))
    begintime= containerdata[0]['timestamp']
    endtime=containerdata[-1]['timestamp']
    return begintime , endtime 


def getpowers(MycontainrId): 
    begin, end = gettimestamps(mycontainerId)
    x=list(sensors.find({'target':'system','timestamp' :{'$gte':begin,'$lte':end}},projection=['rapl','timestamp']))
    conso= pd.DataFrame(x)
    sonde=next(iter(x[0]['rapl']['0']))
    conso['power']=conso['rapl'].apply(lambda row :math.ldexp( row['0'][sonde]['RAPL_ENERGY_PKG'],-32))
    return conso.loc[:,['timestamp','power']]

def getenergy(powers):
    return powers['power'].sum()


In [58]:
testname="calico160119" # the mahcine used for the test 
client = pymongo.MongoClient('172.16.45.8', 27017) # the Ip addresse of the database
db = client.rapls # name of the database [dbname in startmachine.sh ]
sensors= db["sensor"+testname]
sensors.create_index([('timestamp',pymongo.ASCENDING)]) # just to accelerate the search requests 

'timestamp_1'

In [62]:
mycontainerId='c98c4f21af271bec1fed4646cdc93a00b855031667b12c57d7c29903a476c045' # the id of the container 
# you can get this id using the following command in the test machine 
# docker inspect --format "{{.Id}}" containername
power = getpowers(mycontainerId) # get an array of the power consumption during the time stamp 
totalenergy= getenergy(power) # to get the energy consumption of the container 

print(str(totalenergy) + ' joules')
print('my container powers')
power


885.20361328125 joules
my container powers


,timestamp,power
0,2019-01-16 09:30:39.928,26.499207
1,2019-01-16 09:30:40.928,35.755920
2,2019-01-16 09:30:41.928,35.680847
3,2019-01-16 09:30:42.928,35.714233
4,2019-01-16 09:30:43.928,36.393127
5,2019-01-16 09:30:44.929,36.174622
6,2019-01-16 09:30:45.929,35.714172
7,2019-01-16 09:30:46.929,35.674805
8,2019-01-16 09:30:47.929,35.678040
9,2019-01-16 09:30:48.929,35.676819
